In [1]:
import os
import glob
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#### Notes  
- Use the latest not ***overlapping SKUs file***
- for ***medtronics*** dont include Medtronic robotic system accessories  
- in ***GP*** dont include pta balloons
- run together ***SIEMENS & IVD***
- Remove specific SKU for ***embolic and THV***? sentinel SKU?

### read data

In [2]:
path = input("Enter Path (The path of the folder which has Raw data files FOLDER): ")
path.replace('\\','\\\\')
rawDataPath=path
all_files = glob.glob(os.path.join(rawDataPath, "*.csv"))
if len(all_files)>0:
    print(str(len(all_files))+" Files Found")
else:
    print("No files found. Please check path")
    
#C:\Users\Kumar.Anurag\OneDrive - GlobalData PLC\Desktop\Tracker - QC\GP\2023\April\Final Files\Extra Files\Raw Data Files

Enter Path (The path of the folder which has Raw data files FOLDER):  C:\Users\Anamitra.Layek\OneDrive - GlobalData PLC\Documents\2025\QC Work\Sample data csv


5 Files Found


### Read data from additional folder (if needed)

In [3]:
#Use this when raw data files are present in multiple folders
path = input("Enter Path (The path of the folder which has Raw data files FOLDER): ")
path.replace('\\','\\\\')
rawDataPath=path
all_files+= glob.glob(os.path.join(rawDataPath, "*.csv"))
if len(all_files)>0:
    print(str(len(all_files))+" Files Found")
else:
    print("No files found. Please check path")

Enter Path (The path of the folder which has Raw data files FOLDER):  


49 Files Found


In [3]:
all_files

['C:\\Users\\Anamitra.Layek\\OneDrive - GlobalData PLC\\Documents\\2025\\QC Work\\Sample data csv\\Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv',
 'C:\\Users\\Anamitra.Layek\\OneDrive - GlobalData PLC\\Documents\\2025\\QC Work\\Sample data csv\\Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv',
 'C:\\Users\\Anamitra.Layek\\OneDrive - GlobalData PLC\\Documents\\2025\\QC Work\\Sample data csv\\CRAS - Raw Data - Feb 2025.csv',
 'C:\\Users\\Anamitra.Layek\\OneDrive - GlobalData PLC\\Documents\\2025\\QC Work\\Sample data csv\\Flow Diverters - Raw Data - Feb 2025.csv',
 'C:\\Users\\Anamitra.Layek\\OneDrive - GlobalData PLC\\Documents\\2025\\QC Work\\Sample data csv\\Peripheral Nerve Repair - Raw Data - Feb 2025.csv']

#### Create overlapping_df which contains rows from every file

In [4]:
overlapping_df=pd.DataFrame()
for filename in all_files:
    print('Importing ',filename.split('\\')[-1])
    try:
        data = pd.read_csv(filename,usecols=["Transaction.ID","SKU"], encoding='latin1', dtype=str)
    except:
        data = pd.read_csv(filename,usecols=["Transaction ID","SKU"], encoding='latin1', dtype=str)
        data = data.rename({"Transaction ID":"Transaction.ID"}, axis=1)
    #pd.read_excel(f,usecols = "A")
    #data["Transaction.ID"].fillna("Nullc", inplace = True)
    data['filename']=filename
    data['filename'] = data['filename'].map(lambda filename: filename.rsplit('\\')[-1])
    data=data[['Transaction.ID','SKU','filename']]
    overlapping_df = pd.concat([overlapping_df,data])
    
overlapping_df = overlapping_df[overlapping_df['Transaction.ID'].notnull()]

Importing  Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv
Importing  Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv
Importing  CRAS - Raw Data - Feb 2025.csv
Importing  Flow Diverters - Raw Data - Feb 2025.csv
Importing  Peripheral Nerve Repair - Raw Data - Feb 2025.csv


In [5]:
print(len(overlapping_df['filename'].unique()))
overlapping_df['filename'].unique()

5


array(['Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv',
       'Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv',
       'CRAS - Raw Data - Feb 2025.csv',
       'Flow Diverters - Raw Data - Feb 2025.csv',
       'Peripheral Nerve Repair - Raw Data - Feb 2025.csv'], dtype=object)

# Run this only for Nephron

In [6]:
overlapping_df

,Transaction.ID,SKU,filename
0,834116421,2MAXCKIT,Aspiration Thrombectomy Catheters - Raw Data -...
1,1431326633,3MAX,Aspiration Thrombectomy Catheters - Raw Data -...
2,1557598562,3MAX,Aspiration Thrombectomy Catheters - Raw Data -...
3,372714582,3MAXC,Aspiration Thrombectomy Catheters - Raw Data -...
4,373203280,3MAXC,Aspiration Thrombectomy Catheters - Raw Data -...
...,...,...,...
103035,2913752254,AG1040,Peripheral Nerve Repair - Raw Data - Feb 2025.csv
103036,2913757021,111270,Peripheral Nerve Repair - Raw Data - Feb 2025.csv
103037,2913757022,AGX215,Peripheral Nerve Repair - Raw Data - Feb 2025.csv
103038,2913758035,CNW4050,Peripheral Nerve Repair - Raw Data - Feb 2025.csv


In [7]:
overlapping_df = overlapping_df[~overlapping_df.filename.str.contains("Robotic System")]

In [8]:
overlapping_df.filename.unique()

array(['Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv',
       'Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv',
       'CRAS - Raw Data - Feb 2025.csv',
       'Flow Diverters - Raw Data - Feb 2025.csv',
       'Peripheral Nerve Repair - Raw Data - Feb 2025.csv'], dtype=object)

### Total rows

In [9]:
len(overlapping_df)

446158

### Save duplicate rows in a separate data frame

In [10]:
overlapping_df

,Transaction.ID,SKU,filename
0,834116421,2MAXCKIT,Aspiration Thrombectomy Catheters - Raw Data -...
1,1431326633,3MAX,Aspiration Thrombectomy Catheters - Raw Data -...
2,1557598562,3MAX,Aspiration Thrombectomy Catheters - Raw Data -...
3,372714582,3MAXC,Aspiration Thrombectomy Catheters - Raw Data -...
4,373203280,3MAXC,Aspiration Thrombectomy Catheters - Raw Data -...
...,...,...,...
103035,2913752254,AG1040,Peripheral Nerve Repair - Raw Data - Feb 2025.csv
103036,2913757021,111270,Peripheral Nerve Repair - Raw Data - Feb 2025.csv
103037,2913757022,AGX215,Peripheral Nerve Repair - Raw Data - Feb 2025.csv
103038,2913758035,CNW4050,Peripheral Nerve Repair - Raw Data - Feb 2025.csv


In [11]:
####RUN THIS
duplicate_new = overlapping_df[overlapping_df.duplicated('Transaction.ID',keep=False)]
print(duplicate_new.count())

Transaction.ID    16
SKU               16
filename          16
dtype: int64


## Run this for bloomberg (THV & Embolic)

In [11]:
#Read all relevant SKUs from file 
thv_df = pd.read_excel(r"C:\Users\Kumar.Anurag\OneDrive - GlobalData PLC\Desktop\overlapping\not overlaping sku list\Not Overlapping SKUS - bloomberg.xlsx")
relevant_bloomberg_skus = thv_df['SKU'].apply(lambda x: str(x).upper()).tolist()

## Run this for medtronic

In [11]:
df_relev_sku = pd.read_excel(r"C:\Users\Kumar.Anurag\OneDrive - GlobalData PLC\Desktop\overlapping\not overlaping sku list\Not Overlapping SKUS - medtronic.xlsx")
relevant_medtronic_skus = df_relev_sku['SKU'].apply(lambda x: str(x).upper()).tolist()

## Run this for GP

In [13]:
df_relev_sku = pd.read_excel(r"C:\Users\Anamitra.Layek\OneDrive - GlobalData PLC\Documents\2025\QC Work\not overlaping sku list\Not Overlapping SKUS - GP.xlsx")
relevant_gp_skus = df_relev_sku['SKU'].apply(lambda x: str(x).upper()).tolist()

## Extra Analysis for Siemens-IVD Overlapping - There are relevant Overlapping SKUs  
#### include these for siemens and IVD

In [8]:
#Read all relevant SKUs from file 
file = pd.read_excel(r"C:\Users\Kumar.Anurag\OneDrive - GlobalData PLC\Desktop\overlapping\not overlaping sku list\Not Overlapping SKUS - Siemens and IVD.xlsx")
relevant_skus = list(file['SKU'].str.upper())  
# hemo = ['88753', '88754','88755','BS88753','BS88754','BS88755','515', '0515', '00515']
# relevant_skus += hemo
more = ['710256910','5189']
relevant_skus += more

## prepare remove_relevant_overlaps dataframe

In [14]:
#Filter overlapping dataframe by removing relevant skus

# only for SIEMENS & IVD
#removed_relevant_overlaps = duplicate_new[~duplicate_new['SKU'].isin(relevant_skus)]

# for THV (BLOOMBERG)
#removed_relevant_overlaps = duplicate_new[~duplicate_new['SKU'].isin(relevant_bloomberg_skus)]

# only for MEDTRONIC
#removed_relevant_overlaps = duplicate_new[~duplicate_new['SKU'].isin(relevant_medtronic_skus)]

# only for GP
removed_relevant_overlaps = duplicate_new[~duplicate_new['SKU'].isin(relevant_gp_skus)]

# for rest
#removed_relevant_overlaps = duplicate_new.copy()

print('Total rows in removed_relevant_overlaps :', len(removed_relevant_overlaps))

Total rows in removed_relevant_overlaps : 16


In [8]:
# run this for spinal fusion trackers (to remove some words from file name)

#removed_relevant_overlaps.filename = removed_relevant_overlaps.filename.apply(lambda x: x[7:])
removed_relevant_overlaps.filename = removed_relevant_overlaps.filename.apply(lambda x: x[:-5])

removed_relevant_overlaps.filename = removed_relevant_overlaps.filename.apply(lambda x: x.replace(" - Raw Data", ""))
removed_relevant_overlaps.filename = removed_relevant_overlaps.filename.apply(lambda x: x.replace("Spinal Fusion - ", ""))


In [15]:
removed_relevant_overlaps.filename.unique()

array(['Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv',
       'Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv',
       'CRAS - Raw Data - Feb 2025.csv',
       'Flow Diverters - Raw Data - Feb 2025.csv',
       'Peripheral Nerve Repair - Raw Data - Feb 2025.csv'], dtype=object)

In [32]:
#removed_relevant_overlaps.to_excel('test1.xlsx')

## Files with overlapping/duplicte records

In [16]:
titles = list(removed_relevant_overlaps['filename'].unique())
titles

['Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv',
 'Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv',
 'CRAS - Raw Data - Feb 2025.csv',
 'Flow Diverters - Raw Data - Feb 2025.csv',
 'Peripheral Nerve Repair - Raw Data - Feb 2025.csv']

In [17]:
total = 0
for i in titles:
    for j in titles:
        if i!=j:
            tx = removed_relevant_overlaps[removed_relevant_overlaps.filename==j]['Transaction.ID'].values
            dfx = removed_relevant_overlaps[removed_relevant_overlaps.filename==i]
            result = dfx[dfx['Transaction.ID'].isin(tx)]
            if len(result) != 0:
                print(i, j , len(result))
                total += len(result)
            
print(total)

Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv 2
Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv 2
CRAS - Raw Data - Feb 2025.csv Flow Diverters - Raw Data - Feb 2025.csv 2
CRAS - Raw Data - Feb 2025.csv Peripheral Nerve Repair - Raw Data - Feb 2025.csv 2
Flow Diverters - Raw Data - Feb 2025.csv CRAS - Raw Data - Feb 2025.csv 2
Flow Diverters - Raw Data - Feb 2025.csv Peripheral Nerve Repair - Raw Data - Feb 2025.csv 2
Peripheral Nerve Repair - Raw Data - Feb 2025.csv CRAS - Raw Data - Feb 2025.csv 2
Peripheral Nerve Repair - Raw Data - Feb 2025.csv Flow Diverters - Raw Data - Feb 2025.csv 2
16


In [18]:
#####After Confirmation, RUN THIS

path = input("Enter Output Path: ")
path.replace('\\','\\\\')
overlap_dict2 = dict()
pair_list  = []
for a in titles:
    all_a_transaction = removed_relevant_overlaps[removed_relevant_overlaps['filename'] == a]
    overlap_dict2[a] = all_a_transaction
    print(a, end="\t")
    print("\tTransactions: "+str(len(all_a_transaction)))
    
    # check for duplicates in same file
    number_duplicates = len(all_a_transaction)-len(all_a_transaction.drop_duplicates())
    #print(number_duplicates)
    duplicate_present = 'no'
    if number_duplicates > 0:
        duplicate_present = 'yes'
    
    # prepare the list of files which has dupliactes of rows from current file
    x = removed_relevant_overlaps[
        (removed_relevant_overlaps['Transaction.ID'].isin(
            all_a_transaction['Transaction.ID']))&(removed_relevant_overlaps['filename'] != a)]
    tt = list(x['filename'].unique()) # filenames which have overlapping rows with current file (a).
    
    # if dupliacte rows present in same file
    if (duplicate_present == 'yes'):
        print(str(number_duplicates),' duplicate rows.')
        #print()
        duplicate_df = all_a_transaction[all_a_transaction.duplicated('Transaction.ID',keep=False)]
        name = a.split(".")[0] +" duplicate rows"
        duplicate_df.to_excel(path+"\\"+name+".xlsx", index = False, header=True)
    
    # if rows overlap with other file
    if len(tt) != 0:
        for t in tt:
            # avoid duplicate excel files generation
            if ([a,t] in pair_list)|([t,a] in pair_list):
                continue
            else:
                pair_list.append([a,t])
            
            
            print("\tWith ", end=" ")
            print(t, end = " ")
            temp = removed_relevant_overlaps[
                (removed_relevant_overlaps['Transaction.ID'].isin(
                all_a_transaction['Transaction.ID']))& (removed_relevant_overlaps['filename'] == t)]

            print(len(temp))
            if a.split("-")[0] != "COVID":
                name = (''.join(a.split('.')[0])) +" with " + (''.join(t.split('.')[0]))  # .split("-")[0]
            else:
                name = (''.join(a.split('.')[0])) +" with " + (''.join(t.split('.')[0]))   # 
            temp.to_excel(path+"\\"+name+".xlsx", index = False, header=True)
            
    #print("\t",*tt, sep="\n")
    print("\n")

Enter Output Path:  C:\Users\Anamitra.Layek\OneDrive - GlobalData PLC\Documents\2025\QC Work\result\2025\February


Aspiration Thrombectomy Catheters - Raw Data - Feb 2025 - Copy.csv		Transactions: 2
	With  Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv 2


Benign Prostate Hyperplasia Devices - Raw Data - Feb 2025.csv		Transactions: 2


CRAS - Raw Data - Feb 2025.csv		Transactions: 4
	With  Flow Diverters - Raw Data - Feb 2025.csv 2
	With  Peripheral Nerve Repair - Raw Data - Feb 2025.csv 2


Flow Diverters - Raw Data - Feb 2025.csv		Transactions: 4
	With  Peripheral Nerve Repair - Raw Data - Feb 2025.csv 2


Peripheral Nerve Repair - Raw Data - Feb 2025.csv		Transactions: 4


